Crawl Data

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [2]:
standings_urls = ["https://fbref.com/en/comps/9/Premier-League-Stats",
    "https://fbref.com/en/comps/11/Serie-A-Stats",
    "https://fbref.com/en/comps/12/La-Liga-Stats",
    "https://fbref.com/en/comps/13/Ligue-1-Stats",
    "https://fbref.com/en/comps/20/Bundesliga-Stats"]

leagues =  pd.read_excel('./../Club_name.xlsx')
match_df = pd.DataFrame(columns=['Team','Venue', 'Opponent', 'Score', 'Attendance', 'Referee'])

In [3]:
leagues.head()

,Premier League,Serie A,La Liga,Ligue 1,Bundesliga
0,Arsenal,Napoli,Barcelona,Paris S-G,Bayern Munich
1,Manchester City,Inter,Real Madrid,Marseille,Union Berlin
2,Manchester Utd,Roma,Real Sociedad,Lens,Dortmund
3,Newcastle Utd,Lazio,AtlÃ©tico Madrid,Monaco,RB Leipzig
4,Tottenham,Atalanta,Rayo Vallecano,Rennes,Eint Frankfurt


In [4]:
def clean(table, league_name):
    t = table[np.logical_and(table['Comp'] == league_name, table['Venue'] == 'Home')]
    t = t[['Venue', 'GF', 'GA', 'Opponent', 'Attendance', 'Referee']]
    t = t.dropna()
    t['GF'], t['GA'], t['Attendance'] = [t[col].astype(np.int32) for col in ['GF', 'GA', 'Attendance']]
    score = ['\'' + t.loc[r, 'GF'].astype(str) + ' - ' +  t.loc[r, 'GA'].astype(str) \
        if t.loc[r, 'Venue'] == 'Home' else '\''+ t.loc[r, 'GA'].astype(str) + ' - ' + t.loc[r, 'GF'].astype(str) \
        for r in t.index ]
    t.insert(4, 'Score', score, True)
    t = t.drop(columns=['GA', 'GF'])
    return t

In [7]:
# use when 
match_df = pd.read_csv('./MatchDB20.csv')
match_df.head()

,Team,Venue,Opponent,Score,Attendance,Referee
0,Arsenal,Home,Leicester City,'4 - 2,60033,Darren England
1,Arsenal,Home,Fulham,'2 - 1,60164,Jarred Gillett
2,Arsenal,Home,Aston Villa,'2 - 1,60012,Robert Jones
3,Arsenal,Home,Tottenham,'3 - 1,60278,Anthony Taylor
4,Arsenal,Home,Liverpool,'3 - 2,60059,Michael Oliver


In [47]:
print(standings_urls[1:])
print(leagues.iloc[:,1:])

['https://fbref.com/en/comps/11/Serie-A-Stats', 'https://fbref.com/en/comps/12/La-Liga-Stats', 'https://fbref.com/en/comps/13/Ligue-1-Stats', 'https://fbref.com/en/comps/20/Bundesliga-Stats']
          Serie A           La Liga        Ligue 1      Bundesliga
0          Napoli         Barcelona      Paris S-G    Union Berlin
1           Inter       Real Madrid      Marseille   Bayern Munich
2            Roma     Real Sociedad           Lens        Dortmund
3           Lazio  AtlÃ©tico Madrid         Monaco      RB Leipzig
4        Atalanta        Villarreal         Rennes  Eint Frankfurt
5           Milan             Betis          Lille        Freiburg
6         Udinese     Athletic Club        Lorient       Wolfsburg
7          Torino    Rayo Vallecano           Lyon      M'Gladbach
8         Bologna           Osasuna           Nice      Leverkusen
9          Empoli          Mallorca  Clermont Foot   Werder Bremen
10          Monza        Celta Vigo          Reims           KÃ¶ln
11  

In [9]:
count = 0
for standings_url, league in zip(standings_urls[1:],leagues.iloc[:, 1:]):
    data = requests.get(standings_url) 
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    standing = pd.read_html(data.text, attrs={'class': 'stats_table'})[0]

    team_urls = [f"https://fbref.com{l}" for l in links]
    for i, (l, team_name) in enumerate((zip(team_urls, leagues[league]))):
        print(count)
        if i == len(team_urls): 
            break     
        d = requests.get(l)
        match = clean(pd.read_html(d.text, match = "Scores & Fixtures")[0], league)
        match.insert(0, 'Team', team_name, True)
        match_df = pd.concat([match_df, match])
        count += 1
    match_df.to_csv(f'./MatchDB{count}.csv', index=False)
match_df.head()

19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96


,Team,Venue,Opponent,Score,Attendance,Referee
0,Arsenal,Home,Leicester City,'4 - 2,60033,Darren England
1,Arsenal,Home,Fulham,'2 - 1,60164,Jarred Gillett
2,Arsenal,Home,Aston Villa,'2 - 1,60012,Robert Jones
3,Arsenal,Home,Tottenham,'3 - 1,60278,Anthony Taylor
4,Arsenal,Home,Liverpool,'3 - 2,60059,Michael Oliver


In [10]:
match_df.describe()

,Attendance
count,1156.000000
mean,33976.012976
std,19005.945956
min,3561.000000
25%,17982.000000
50%,30826.000000
75%,48396.000000
max,92605.000000


### When finishing dataFrames

In [11]:
match_df.insert(0, 'match_id', list(range(1,len(match_df.index)+1)), False)
match_df = match_df.drop(columns=['Venue'])
match_df.head()

,match_id,Team,Opponent,Score,Attendance,Referee
0,1,Arsenal,Leicester City,'4 - 2,60033,Darren England
1,2,Arsenal,Fulham,'2 - 1,60164,Jarred Gillett
2,3,Arsenal,Aston Villa,'2 - 1,60012,Robert Jones
3,4,Arsenal,Tottenham,'3 - 1,60278,Anthony Taylor
4,5,Arsenal,Liverpool,'3 - 2,60059,Michael Oliver


In [12]:
club_df = pd.read_csv('../club/clubDB.csv',)
club_df.head()

,club_id,club_name,nation,stadium_id
0,1,Manchester Utd,ENG,1
1,2,West Ham,ENG,2
2,3,Tottenham,ENG,3
3,4,Arsenal,ENG,4
4,5,Manchester City,ENG,5


In [13]:
print(club_df)

    club_id        club_name nation  stadium_id
0         1   Manchester Utd    ENG           1
1         2         West Ham    ENG           2
2         3        Tottenham    ENG           3
3         4          Arsenal    ENG           4
4         5  Manchester City    ENG           5
..      ...              ...    ...         ...
93       94       Leverkusen    GER          94
94       95       Hoffenheim    GER          95
95       96        Wolfsburg    GER          96
96       97           Bochum    GER          97
97       98     Union Berlin    GER          98

[98 rows x 4 columns]


In [14]:
team = list(club_df['club_name'])
id = list(club_df['club_id'])
match_df['Team'] = match_df['Team'].replace(team, id)
match_df['Opponent'] = match_df['Opponent'].replace(team, id)
match_df.head()

,match_id,Team,Opponent,Score,Attendance,Referee
0,1,4,13,'4 - 2,60033,Darren England
1,2,4,18,'2 - 1,60164,Jarred Gillett
2,3,4,8,'2 - 1,60012,Robert Jones
3,4,4,3,'3 - 1,60278,Anthony Taylor
4,5,4,6,'3 - 2,60059,Michael Oliver


In [15]:
match_df.to_csv('./MatchDB.csv', index=False, header=False)

In [16]:
print(match_df)

    match_id Team  Opponent   Score  Attendance              Referee
0          1    4        13  '4 - 2       60033       Darren England
1          2    4        18  '2 - 1       60164       Jarred Gillett
2          3    4         8  '2 - 1       60012         Robert Jones
3          4    4         3  '3 - 1       60278       Anthony Taylor
4          5    4         6  '3 - 2       60059       Michael Oliver
..       ...  ...       ...     ...         ...                  ...
13      1152   84        91  '0 - 2       61115    Christian Dingert
15      1153   84        92  '1 - 0       58256       Tobias Reichel
16      1154   84        82  '0 - 2       62271         Felix Zwayer
18      1155   84        89  '1 - 6       56841      Benjamin Cortus
19      1156   84        88  '0 - 0       61571  Matthias Jöllenbeck

[1156 rows x 6 columns]


In [21]:
m = pd.DataFrame(columns=['Team','Venue', 'Opponent', 'Score', 'Attendance', 'Referee'])
m = pd.read_csv('./MatchDB.csv', header=None)
m.head()

,0,1,2,3,4,5
0,1,4,13,'4 - 2,60033,Darren England
1,2,4,18,'2 - 1,60164,Jarred Gillett
2,3,4,8,'2 - 1,60012,Robert Jones
3,4,4,3,'3 - 1,60278,Anthony Taylor
4,5,4,6,'3 - 2,60059,Michael Oliver


In [24]:
print(m.columns)
m[0] = list(range(1, len(m.index) + 1))

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')


In [28]:
print(m)
m.to_csv('./back_up.csv', index=False)

       0   1   2       3      4                    5
0      1   4  13  '4 - 2  60033       Darren England
1      2   4  18  '2 - 1  60164       Jarred Gillett
2      3   4   8  '2 - 1  60012         Robert Jones
3      4   4   3  '3 - 1  60278       Anthony Taylor
4      5   4   6  '3 - 2  60059       Michael Oliver
..   ...  ..  ..     ...    ...                  ...
973  974  84  91  '0 - 2  61115    Christian Dingert
974  975  84  92  '1 - 0  58256       Tobias Reichel
975  976  84  82  '0 - 2  62271         Felix Zwayer
976  977  84  89  '1 - 6  56841      Benjamin Cortus
977  978  84  88  '0 - 0  61571  Matthias Jöllenbeck

[978 rows x 6 columns]


In [31]:
m.insert(3, 'lyear_id', 1, True)
m.head()

,0,1,2,lyear_id,3,4,5
0,1,4,13,1,'4 - 2,60033,Darren England
1,2,4,18,1,'2 - 1,60164,Jarred Gillett
2,3,4,8,1,'2 - 1,60012,Robert Jones
3,4,4,3,1,'3 - 1,60278,Anthony Taylor
4,5,4,6,1,'3 - 2,60059,Michael Oliver


In [34]:
m['lyear_id'][209:391] = 125
m['lyear_id'][391:591] = 160
m['lyear_id'][591:811]= 195
m['lyear_id'][811:] = 223

C:\Users\DELL\AppData\Local\Temp\ipykernel_14064\2038517585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['lyear_id'][209:391] = 125
C:\Users\DELL\AppData\Local\Temp\ipykernel_14064\2038517585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['lyear_id'][391:591] = 160
C:\Users\DELL\AppData\Local\Temp\ipykernel_14064\2038517585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['lyear_id'][591:811]= 195
C:\Users\DELL\App

In [36]:
m.to_csv('./back_up1.csv', index=False)